In [ ]:
"""
    value_iteration(
        return_function,  # Return for transitioning from current to next state
        state_space,      # Vector of possible states
        discount_factor;  # Discount factor β ∈ (0,1)
        tolerance=1e-6,
        max_iterations=1000,
        initial_value_function=Dict(s => 0.0 for s in state_space),
        indicator=false
    )

Performs value function iteration for a problem where agent directly chooses next state.

Parameters:
- return_function: Function(current_state, next_state) -> reward
- state_space: Vector of possible states
- discount_factor: Discount factor β ∈ (0,1)
- tolerance: Convergence tolerance
- max_iterations: Maximum number of iterations
- initial_value_function: Initial value function
- capture: capture the value function and policy at certain iterations

Returns:
- value_function: Dictionary mapping states to values
- policy: Dictionary mapping states to optimal next states
- iterations: Number of iterations until convergence
- diffs: Array of iteration-wise maximum values of change in value function
- pdiffs: Array of iteration-wise maximum values of change in policy function
"""
function value_iteration(
    return_function,
    state_space,
    discount_factor;
    tolerance=1e-6,
    max_iterations=1000,
    initial_value_function=Dict(s => 0.0 for s in state_space),
    indicator=false
)
    
    diffs = Float64[] # track differences
    pdiffs = Float64[] # track differences in policy
    captures = []

    # Initialize value function
    value_function = initial_value_function
    policy = Dict(s => s for s in state_space)  # Initial policy: stay at current state
    
    for iter in 1:max_iterations
        max_diff = 0.0
        max_policy_diff = 0.0
        value_function_new = Dict()
        oldpolicy = copy(policy)
        
        # Update value for each current state
        for current_state in state_space
            # Find maximum value over all possible next states
            values = Float64[]
            
            # Try each possible next state
            for next_state in state_space
                # Calculate value of choosing this next state:
                # Current reward + discounted future value
                value = return_function(current_state, next_state) + 
                       discount_factor * value_function[next_state]
                push!(values, value)
            end
            
            # Update value function and policy
            max_value, max_index = findmax(values)
            value_function_new[current_state] = max_value
            policy[current_state] = state_space[max_index]
            
            # Track maximum change
            max_diff = max(max_diff, abs(value_function_new[current_state] - 
                                       value_function[current_state]))
            max_policy_diff = max(max_policy_diff, abs(log(policy[current_state]) - log(oldpolicy[current_state])))
        end

        push!(diffs, max_diff)
        push!(pdiffs, max_policy_diff)
        if indicator
            if iter in [100, 200, 300, 400]
                push!(captures, (iter, copy(value_function_new)))
            end
        end
        
        # Check for convergence
        if max_diff < tolerance
            return value_function_new, policy, iter, diffs, pdiffs, captures
        end
        
        value_function = value_function_new
    end
    
    @warn "Maximum iterations reached without convergence"
    return value_function, policy, max_iterations, diffs, captures
end

value_iteration

In [ ]:
function rck(k, k_next)
    # production function: f(k) = Ak^α
    α = 0.33
    A = 1
    δ = 0.1
    
    # Current production
    production = A*k^α
    
    # Investment needed to reach k_next
    investment = k_next - (1-δ)*k  # δ is depreciation rate
    
    # Consumption is production minus investment
    consumption = production - investment
    
    # Return negative infinity for infeasible transitions
    if consumption <= 0
        return -Inf
    end
    
    # Utility from consumption
    return log(consumption)
end

rck (generic function with 1 method)

In [ ]:
# Set up and solve the problem
k_min_1, k_max_1 = (0.1 * 3.53287891716), 3.53287891716
state_space_1 = range(k_min_1, k_max_1, length=401)
discount_factor = 0.96

# Solve the problem
value_function_1, policy_1, iterations_1, diffs_1, pdiffs_1, captures_dump = value_iteration(
    rck,
    state_space_1,
    discount_factor
)

(Dict{Any, Any}(0.78253268015094 => 0.09049204883305673, 3.49313402934195 => 3.7468553813326526, 2.8969607120712 => 3.1763820709244457, 0.8222775679689901 => 0.18327451954830976, 1.02100200705924 => 0.607950330544671, 1.35485906473086 => 1.2136684452684265, 2.3723281928729403 => 2.6078345023124125, 1.7364099877841401 => 1.797039725672825, 0.57585926349708 => -0.45079843129220115, 1.6012773692027702 => 1.6008670568205297…), Dict(0.78253268015094 => 1.0845938275681202, 3.49313402934195 => 3.50108300690556, 2.8969607120712 => 2.98439946527091, 0.8222775679689901 => 1.1243387153861701, 1.02100200705924 => 1.3071651993492002, 1.35485906473086 => 1.61717532432999, 2.3723281928729403 => 2.53130774414514, 1.7364099877841401 => 1.96693033712883, 0.57585926349708 => 0.8779204109142601, 1.6012773692027702 => 1.8476956736746801…), 270, [1.4685522456868598, 0.478922162669023, 0.2834249170819234, 0.2244711050115944, 0.17422995916456463, 0.13334601393715362, 0.10059033757508207, 0.07468960152708726, 

In [ ]:
# Set up and solve the problem
k_min_2, k_max_2 = 3.53287891716, 5.9415725271
state_space_2 = range(k_min_2, k_max_2, length=101)
discount_factor = 0.96

# Solve the problem
value_function_2, policy_2, iterations_2, diffs_2, pdiffs_2, captures_dump = value_iteration(
    rck,
    state_space_2,
    discount_factor
)

(Dict{Any, Any}(5.9174855910006 => 5.5710970111738245, 3.9905307030486 => 4.175993876208065, 5.5561815495096 => 5.334936846120469, 5.459833805112 => 5.2703508674281645, 4.5686171694342 => 4.63355101024412, 4.8094865304282 => 4.813065430675243, 5.4116599329132 => 5.237764856458238, 4.3036608723408 => 4.428802117302365, 3.8219221503528003 => 4.034497717666469, 4.0627915113468 => 4.235456621547348…), Dict(5.9174855910006 => 5.5561815495096, 3.9905307030486 => 3.9182698947504, 5.5561815495096 => 5.2671383163168, 5.459833805112 => 5.1707905719192, 4.5686171694342 => 4.4240955528378, 4.8094865304282 => 4.616791041633, 5.4116599329132 => 5.1226166997204, 4.3036608723408 => 4.1832261918438, 3.8219221503528003 => 3.773748278154, 4.0627915113468 => 3.9905307030486…), 294, [1.2850954187010066, 0.4359280135367336, 0.2801741905818871, 0.21440319291668164, 0.17873203380875857, 0.156297713678768, 0.14078971975398957, 0.12914413251501422, 0.1199483531712362, 0.11240804956706762  …  1.3960216387332025e

In [ ]:
print(iterations_1, "\n", iterations_2)

270
294

In [ ]:
plot(diffs_1[1:iterations_1], label="Γ_1", title="Convergence Rates for Γ_1 and Γ_2", xlabel="Iterations", ylabel="Maximum of Value Function Difference")
plot!(diffs_2[1:iterations_2], label="Γ_2")

UndefVarError: UndefVarError: `plot` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
plot(pdiffs_1[1:iterations_1], label="Γ_1", title="Convergence Rates for Γ_1 and Γ_2", xlabel="Iterations", ylabel="Maximum of Log of Policy Function Difference")
plot!(pdiffs_2[1:iterations_2], label="Γ_2")

UndefVarError: UndefVarError: `plot` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
w_1 = Dict(s => (((1-0.96)/0.96) * log(s^(0.33) -0.1s)) for s in state_space_1)
w_2 = Dict(s => (((1-0.96)/0.96) * log(s^(0.33) -0.1s)) for s in state_space_2)

Dict{Float64, Float64} with 101 entries:
  5.91749 => 0.00781546
  3.99053 => 0.00688917
  5.55618 => 0.00778555
  5.45983 => 0.0077682
  4.56862 => 0.0073903
  4.80949 => 0.00753465
  5.41166 => 0.00775796
  4.30366 => 0.0071897
  3.82192 => 0.00669545
  4.06279 => 0.0069651
  3.96644 => 0.00686293
  4.47227 => 0.00732266
  5.8934  => 0.00781512
  4.83357 => 0.00754723
  5.07444 => 0.00765564
  5.53209 => 0.00778159
  4.08688 => 0.0069895
  4.0387  => 0.00694024
  4.35183 => 0.00722964
  ⋮       => ⋮

In [ ]:
# Solve the problem
value_function_3, policy_3, iterations_3, diffs_3, pdiffs_3, captures_dump = value_iteration(
    rck,
    state_space_1,
    discount_factor,
    initial_value_function=w_1
)

In [ ]:
# Solve the problem
value_function_4, policy_4, iterations_4, diffs_4, pdiffs_4, captures_dump = value_iteration(
    rck,
    state_space_2,
    discount_factor,
    initial_value_function=w_2
)

In [ ]:
print(iterations_3, "\n", iterations_4)

In [ ]:
plot(diffs_3[1:iterations_3], label="Γ_1", title="Convergence Rates for Γ_1 and Γ_2", xlabel="Iterations", ylabel="Maximum of Value Function Difference")
plot!(diffs_4[1:iterations_4], label="Γ_2")

In [ ]:
plot(pdiffs_3[1:iterations_3], label="Γ_1", title="Convergence Rates for Γ_1 and Γ_2", xlabel="Iterations", ylabel="Maximum of Log of Policy Function Difference")
plot!(pdiffs_4[1:iterations_4], label="Γ_2")

In [ ]:
w_3 = Dict(s => value_function_1[3.53287891716] for s in state_space_1)
w_4 = Dict(s => (((1-0.96)/0.96) * log(s^(0.33) -0.1s) + (s-3.53287891716)) for s in state_space_2)

In [ ]:
# Solve the problem
value_function_5, policy_5, iterations_5, diffs_5, pdiffs_5, captures_5 = value_iteration(
    rck,
    state_space_1,
    discount_factor,
    initial_value_function=w_3,
    indicator=true
)

In [ ]:
# Solve the problem
value_function_6, policy_6, iterations_6, diffs_6, pdiffs_6, captures_6 = value_iteration(
    rck,
    state_space_2,
    discount_factor,
    initial_value_function=w_4,
    indicator=true
)

In [ ]:
print(iterations_5, "\n", iterations_6)

In [ ]:
plot(diffs_5[1:iterations_5], label="Γ_1", title="Convergence Rates for Γ_1 and Γ_2", xlabel="Iterations", ylabel="Maximum of Value Function Difference")
plot!(diffs_6[1:iterations_6], label="Γ_2")

In [ ]:
plot(pdiffs_5[1:iterations_5], label="Γ_1", title="Convergence Rates for Γ_1 and Γ_2", xlabel="Iterations", ylabel="Maximum of Log of Policy Function Difference")
plot!(pdiffs_6[1:iterations_6], label="Γ_2")

In [ ]:
for i in 1:length(captures_5)
    plot(state_space_1, collect(values(captures_5[i][2])), label="Γ_1 at Iteration $(captures_5[i][1])", title="Value Function Comparison", xlabel="State", ylabel="Value Function")
    plot!(state_space_2, collect(values(captures_6[i][2])), label="Γ_2 at Iteration $(captures_6[i][1])")
end

In [ ]:
plot(state_space_1, collect(values(captures_5[1][2])), label="Γ_1 at Iteration $(captures_5[1][1])", title="Value Function Comparison", xlabel="State", ylabel="Value Function")
plot!(state_space_2, collect(values(captures_6[1][2])), label="Γ_2 at Iteration $(captures_6[1][1])")

In [ ]:
plot(state_space_1, collect(values(captures_5[2][2])), label="Γ_1 at Iteration $(captures_5[2][1])", title="Value Function Comparison", xlabel="State", ylabel="Value Function")
plot!(state_space_2, collect(values(captures_6[2][2])), label="Γ_2 at Iteration $(captures_6[2][1])")

In [ ]:
function value_iteration_skip(
    return_function,
    state_space,
    discount_factor;
    tolerance=1e-6,
    max_iterations=1000,
    initial_value_function=Dict(s => 0.0 for s in state_space),
    indicator=false
)
    
    diffs = Float64[] # track differences
    pdiffs = Float64[] # track differences in policy
    captures = []

    # Initialize value function
    value_function = initial_value_function
    policy = Dict(s => s for s in state_space)  # Initial policy: stay at current state
    
    for iter in 1:max_iterations
        max_diff = 0.0
        max_policy_diff = 0.0
        value_function_new = Dict()
        oldpolicy = copy(policy)
        
        # Update value for each current state
        for current_state in state_space
            # Find maximum value over all possible next states
            values = Float64[]
            
            # Try each possible next state
            for next_state in state_space
                # Calculate value of choosing this next state:
                # Current reward + discounted future value
                value = return_function(current_state, next_state) + 
                       discount_factor * value_function[next_state]
                push!(values, value)
            end
            
            # Update value function
            max_value, max_index = findmax(values)
            value_function_new[current_state] = max_value
            
            # Update policy every 2 iterations
            if iter % 2 == 0
                policy[current_state] = state_space[max_index]
            end
            
            # Track maximum change
            max_diff = max(max_diff, abs(value_function_new[current_state] - 
                                       value_function[current_state]))
            if iter % 2 == 0
                max_policy_diff = max(max_policy_diff, abs(log(policy[current_state]) - log(oldpolicy[current_state])))
            end
        end

        push!(diffs, max_diff)
        if iter % 2 == 0
            push!(pdiffs, max_policy_diff)
        end
        if indicator
            if iter in [100, 200, 300, 400]
                push!(captures, (iter, copy(value_function_new)))
            end
        end
        
        # Check for convergence
        if max_diff < tolerance
            return value_function_new, policy, iter, diffs, pdiffs, captures
        end
        
        value_function = value_function_new
    end
    
    @warn "Maximum iterations reached without convergence"
    return value_function, policy, max_iterations, diffs, captures
end

In [ ]:
value_function_7, policy_7, iterations_7, diffs_7, pdiffs_7, captures_dump = value_iteration_skip(
    rck,
    state_space_1,
    discount_factor
)

In [ ]:
value_function_8, policy_8, iterations_8, diffs_8, pdiffs_8, captures_dump = value_iteration_skip(
    rck,
    state_space_2,
    discount_factor
)

In [ ]:
print(iterations_7, "\n", iterations_8)

In [ ]:
plot(diffs_7[1:iterations_7], label="Γ_1", title="Convergence Rates for Γ_1 and Γ_2", xlabel="Iterations", ylabel="Maximum of Value Function Difference")
plot!(diffs_8[1:iterations_8], label="Γ_2")

In [ ]:
plot(pdiffs_7[1:135], label="Γ_1", title="Convergence Rates for Γ_1 and Γ_2", xlabel="Iterations", ylabel="Maximum of Log of Policy Function Difference")
plot!(pdiffs_8[1:147], label="Γ_2")

In [ ]:
"""
    value_iteration_stochastic(
        return_function,  # Return for transitioning from current to next state
        state_space,      # Vector of possible states
        shock_space,      # Vector of possible shocks
        transition_probabilities,  # Matrix of transition probabilities
        discount_factor;  # Discount factor β ∈ (0,1)
        tolerance=1e-6,
        max_iterations=1000,
        initial_value_function=Dict((s, z) => 0.0 for s in state_space for z in shock_space)
    )

Performs value function iteration for a problem with stochastic transitions.

Parameters:
- return_function: Function(current_state, shock, next_state) -> reward
- state_space: Vector of possible states
- shock_space: Vector of possible shocks
- transition_probabilities: Matrix of transition probabilities
- discount_factor: Discount factor β ∈ (0,1)
- tolerance: Convergence tolerance
- max_iterations: Maximum number of iterations
- initial_value_function: Initial value function

Returns:
- value_function: Dictionary mapping (state, shock) to values
- policy: Dictionary mapping (state, shock) to optimal next states
- iterations: Number of iterations until convergence
- diffs: Array of iteration-wise maximum values of change in value function
"""
function value_iteration_stochastic(
    return_function,
    state_space,
    shock_space,
    transition_probabilities,
    discount_factor;
    tolerance=1e-1,
    max_iterations=1000,
    initial_value_function=Dict((s, z) => 0.0 for s in state_space for z in shock_space)
)
    
    diffs = Float64[] # track differences

    # Initialize value function
    value_function = initial_value_function
    policy = Dict((s, z) => s for s in state_space for z in shock_space)  # Initial policy: stay at current state
    
    for iter in 1:max_iterations
        max_diff = 0.0
        value_function_new = Dict()
        
        # Update value for each current state and shock
        for current_state in state_space
            for current_shock in shock_space
                # Find expected value over all possible next states
                values = Float64[]
                
                # Try each possible next state
                for next_state in state_space
                    # Calculate expected value over all possible next shocks
                    expected_value = sum(transition_probabilities[current_shock, next_shock] * value_function[(next_state, next_shock)] for next_shock in shock_space)
                    
                    # Calculate value of choosing this next state:
                    # Current reward + discounted expected future value
                    value = return_function(current_state, current_shock, next_state) + discount_factor * expected_value
                    push!(values, value)
                end
                
                # Update value function and policy
                max_value, max_index = findmax(values)
                value_function_new[(current_state, current_shock)] = max_value
                policy[(current_state, current_shock)] = state_space[max_index]
                
                # Track maximum change
                max_diff = max(max_diff, abs(value_function_new[(current_state, current_shock)] - 
                                           value_function[(current_state, current_shock)]))
            end
        end

        push!(diffs, max_diff)
        
        # Check for convergence
        if max_diff < tolerance
            return value_function_new, policy, iter, diffs
        end
        
        value_function = value_function_new
    end
    
    @warn "Maximum iterations reached without convergence"
    return value_function, policy, max_iterations, diffs
end

value_iteration_stochastic

In [3]:
function stoch_rck(k, z, k_next)
    # production function: f(k) = Ak^α
    α = 0.33
    A = 1
    δ = 0.1
    
    # Current production
    production = ℯ^z * A*k^α
    
    # Investment needed to reach k_next
    investment = k_next - (1-δ)*k  # δ is depreciation rate
    
    # Consumption is production minus investment
    consumption = production - investment
    
    # Return negative infinity for infeasible transitions
    if consumption <= 0
        return -Inf
    end
    
    # Utility from consumption
    return log(consumption)
end

stoch_rck (generic function with 1 method)

In [ ]:
# Set up and solve the problem
k_min_stoch, k_max_stoch = (0.8 * 3.53287891716), (1.2 * 3.53287891716)
state_space_stoch = range(k_min_stoch, k_max_stoch, length=501)
discount_factor = 0.96

min_shock, max_shock = -0.2, 0.2
shock_space = range(min_shock, max_shock, length=201)
shock_space = exp.(shock_space)
probs = Dict((shock_space[i], shock_space[j]) => 1/201 for i in 1:length(shock_space) for j in 1:length(shock_space))

# Solve the problem
value_function_stoch, policy_stoch, iterations_stoch, diffs_stoch = value_iteration_stochastic(
    stoch_rck,
    state_space_stoch,
    shock_space,
    probs,
    discount_factor
)

In [2]:
function value_iteration_stochastic_new(
    return_function,            # Reward function: return_function(current_state, shock, next_state) -> reward
    state_space,                # Vector of possible states
    shock_space,                # Vector of possible shocks
    transition_probabilities,   # Matrix of transition probabilities
    discount_factor;            # Discount factor β ∈ (0,1)
    tolerance=1e-6,             # Convergence tolerance
    max_iterations=1000,        # Maximum number of iterations
    initial_value_function=nothing # Initial value function (optional)
)
    # Map states and shocks to indices
    state_to_index = Dict(state_space[i] => i for i in 1:length(state_space))
    shock_to_index = Dict(shock_space[j] => j for j in 1:length(shock_space))
    
    # Initialize value function and policy
    S, Z = length(state_space), length(shock_space)
    value_function = initial_value_function !== nothing ? initial_value_function : zeros(S, Z)
    policy = zeros(Int, S, Z)  # Policy maps to indices of state_space
    diffs = Float64[]          # Track differences for convergence

    for iter in 1:max_iterations
        max_diff = 0.0
        value_function_new = copy(value_function)

        # Iterate over all states and shocks
        for current_state_idx in 1:S
            for current_shock_idx in 1:Z
                values = zeros(Float64, S)

                # Compute value for all possible next states
                for next_state_idx in 1:S
                    # Expected value over all shocks
                    expected_value = sum(
                        transition_probabilities[(shock_space[current_shock_idx], shock_space[next_shock_idx])] *
                        value_function[next_state_idx, next_shock_idx]
                        for next_shock_idx in 1:Z
                    )
                    # Total value = immediate reward + discounted future value
                    values[next_state_idx] = return_function(
                        state_space[current_state_idx],
                        shock_space[current_shock_idx],
                        state_space[next_state_idx]
                    ) + discount_factor * expected_value
                end

                # Find the maximum value and corresponding state
                max_value, max_index = findmax(values)
                value_function_new[current_state_idx, current_shock_idx] = max_value
                policy[current_state_idx, current_shock_idx] = max_index

                # Update the maximum difference for convergence check
                max_diff = max(max_diff, abs(max_value - value_function[current_state_idx, current_shock_idx]))
            end
        end

        # Track the maximum difference for each iteration
        push!(diffs, max_diff)
        value_function = value_function_new

        # Check for convergence
        if max_diff < tolerance
            return Dict((state_space[i], shock_space[j]) => value_function[i, j] for i in 1:S for j in 1:Z),
                   Dict((state_space[i], shock_space[j]) => state_space[policy[i, j]] for i in 1:S for j in 1:Z),
                   iter,
                   diffs
        end
    end

    # Warn if convergence was not achieved within max_iterations
    @warn "Maximum iterations reached without convergence"
    return Dict((state_space[i], shock_space[j]) => value_function[i, j] for i in 1:S for j in 1:Z),
           Dict((state_space[i], shock_space[j]) => state_space[policy[i, j]] for i in 1:S for j in 1:Z),
           max_iterations,
           diffs
end



value_iteration_stochastic_new (generic function with 1 method)

In [4]:
# Set up and solve the problem
k_min_stoch, k_max_stoch = (0.8 * 3.53287891716), (1.2 * 3.53287891716)
state_space_stoch = range(k_min_stoch, k_max_stoch, length=501)
discount_factor = 0.96

min_shock, max_shock = -0.2, 0.2
shock_space = range(min_shock, max_shock, length=201)
shock_space = exp.(shock_space)
probs = Dict((shock_space[i], shock_space[j]) => 1/201 for i in 1:length(shock_space) for j in 1:length(shock_space))

# Solve the problem
value_function_stoch, policy_stoch, iterations_stoch, diffs_stoch = value_iteration_stochastic_new(
    stoch_rck,
    state_space_stoch,
    shock_space,
    probs,
    discount_factor
)

InterruptException: InterruptException:

In [1]:
using Base.Threads

# Check the current number of threads
println("Current number of threads: ", nthreads())

Current number of threads: 8


In [ ]:
function value_iteration_stochastic_new_parallel(
    model, state_space, shock_space, probs, discount_factor;
    max_iter=1000, tol=1e-6
)
    V = zeros(length(state_space), length(shock_space))
    policy = zeros(length(state_space), length(shock_space))
    diff = Inf
    iter = 0

    while diff > tol && iter < max_iter
        V_new = copy(V)
        @threads for i in 1:length(state_space)
            for j in 1:length(shock_space)
                v_max = -Inf
                expected_value = 0.0
                for k in 1:length(shock_space)
                    expected_value += probs[(shock_space[j], shock_space[k])] * V[i, k]
                end
                value = model(state_space[i], shock_space[j], expected_value) + discount_factor * expected_value
                if value > v_max
                    v_max = value
                    policy[i, j] = shock_space[j]
                end
                V_new[i, j] = v_max
            end
        end
        diff = maximum(abs.(V_new .- V))
        V = V_new
        iter += 1
    end

    return V, policy, iter, diff
end

value_iteration_stochastic_new_parallel (generic function with 1 method)

In [16]:

# Set up and solve the problem
k_min_stoch, k_max_stoch = (0.8 * 3.53287891716), (1.2 * 3.53287891716)
state_space_stoch = range(k_min_stoch, k_max_stoch, length=501)
discount_factor = 0.96

min_shock, max_shock = -0.2, 0.2
shock_space = range(min_shock, max_shock, length=201)
shock_space = exp.(shock_space)
probs = Dict((shock_space[i], shock_space[j]) => 1/201 for i in 1:length(shock_space) for j in 1:length(shock_space))

# Solve the problem using the parallel version
value_function_stoch, policy_stoch, iterations_stoch, diffs_stoch = value_iteration_stochastic_new_parallel(
    stoch_rck,
    state_space_stoch,
    shock_space,
    probs,
    discount_factor
)

# Convert arrays to dictionaries
value_function_dict = Dict((state_space_stoch[i], shock_space[j]) => value_function_stoch[i, j] for i in 1:length(state_space_stoch) for j in 1:length(shock_space))
policy_function_dict = Dict((state_space_stoch[i], shock_space[j]) => policy_stoch[i, j] for i in 1:length(state_space_stoch) for j in 1:length(shock_space))

Dict{Tuple{Float64, Float64}, Float64} with 100701 entries:
  (3.63463, 0.974335) => 0.974335
  (3.50179, 1.18057)  => 1.18057
  (3.52157, 0.878095) => 0.878095
  (3.65158, 1.09856)  => 1.09856
  (3.40004, 0.841979) => 0.841979
  (3.5781, 0.921272)  => 0.921272
  (4.23098, 0.838618) => 0.838618
  (3.28416, 0.904837) => 0.904837
  (3.16546, 0.921272) => 0.921272
  (3.01567, 0.852144) => 0.852144
  (3.23046, 1.16416)  => 1.16416
  (3.86921, 0.879853) => 0.879853
  (4.20554, 0.908464) => 0.908464
  (3.40852, 1.08981)  => 1.08981
  (3.77594, 0.841979) => 0.841979
  (3.2022, 1.17821)   => 1.17821
  (3.21916, 1.123)    => 1.123
  (3.65158, 0.96464)  => 0.96464
  (3.20503, 1.14798)  => 1.14798
  ⋮                   => ⋮

In [ ]:
using Plots
plotlyjs()

# Create a 3D plot
x = repeat(state_space_stoch, outer=length(shock_space))
y = repeat(shock_space, inner=length(state_space_stoch))
z = vec(value_function_stoch)

plot(x, y, z, st=:surface, title="Value Function for Stochastic RCK Model", xlabel="State", ylabel="Shock", zlabel="Value")

In [18]:
import Pkg; Pkg.add("PlotlyJS")